# Chapter 16. Finding, Counting, and Listing All Triangles in Large Graphs

[그래프 이론 - 위키피디아](https://ko.wikipedia.org/wiki/%EA%B7%B8%EB%9E%98%ED%94%84_%EC%9D%B4%EB%A1%A0)
- 유한개의 노드인 vertice와 노드간의 연결인 edges로 이루어짐.
![](https://upload.wikimedia.org/wikipedia/commons/thumb/5/5b/6n-graf.svg/333px-6n-graf.svg.png)

Triangles count 하기 전에 triad 와 triangle 을 정의하자.
    - G( graph )에서 3개의 다른 node,  T= (a, b, c)가 있다고 하자.
    - 만약, 2개의 노드가 연결( {(a,b), (a,c)} ) 되어 있으면, T 를 triad 라 한다.
    - 만약, 3개의 노드가 연결( {(a,b), (a,c), (b,c)} )되어 있으면, triangle 이라 한다.
    
그래프 분석에서,  3개의 중요한 metrics가 있음.
    - Global clustering coefficient
    - Transitivity ratio, defined as
![](chap16_01.jpg)
    - Local clustering coefficient

이 3가지 metrics을 계산하기 위해서는, 그래프에서 triangles의 개수를 알 필요가 있음.

그래프에서 triangle을 찾고, 세고, 리스트를 뽑는 것을 2가지 MapReduce solutions을 제공함
- MapReduce/Hadoop solution은 3개의 분리된 MapReduce job으로 구성됨.
- Spark solution은 map()와 reduce() 와 같은 고차함수를 이용해서 하나의 Driver로 구성되고, Spark는 GraphX라는 graph-parallel API가 제공됨.


## Basic Graph Concepts

- V 는 유한개의 노드의 집합이라고 하자.
- E 는 노드간의 연결인  edge의 유한개의 집합라고 하자. 
- 비방향성 그래프를  G = (V, E) 라고 정의함.
- 노드의 개수를 n,  edge의 개수를 m 이라고 함.
- v의 차수, d(v)는 v와 인접해 있는 노드들의 개수로 정의함.
    - d( 1 ) =  ?
    - d( 2 ) =  ?
    - d( 3 ) =  ?
- 아래 그래프에서  triangles의 개수는 2개.
    - {{2, 3}, {3, 4}, {4, 2}}
    - {{2, 4}, {4, 5}, {5, 2}}
![](chap16_02.jpg)

## Importance of Counting Triangles

- 2개의 중요한 metrics (clustering coefficients and transitive ratios)을 측정에 필요.
- Social graphs에서 침입탐지, communities 와 spamming 식별에 사용됨.
- biological networks에서는 protein–protein interaction networks을 찾는데 도움을 줌.
- triangles와 high clustering coefficient은 social, biological, web와 other networks의 중요한 특성을 나타냄.

## MapReduce/Hadoop Solution

### Step 1: MapReduce in Action

![](chap16_03.jpg)

![](chap16_04.jpg)

### Step 2: Identify Triangles

![](chap16_05.jpg)

### Step 3: Remove Duplicate Triangles

Mapper
```
(1, 2, 3) will emit (1, 2, 3)
(1, 3, 2) will emit (1, 2, 3)
(2, 1, 3) will emit (1, 2, 3)
(2, 3, 1) will emit (1, 2, 3)
(3, 1, 2) will emit (1, 2, 3)
(3, 2, 1) will emit (1, 2, 3)
```

Reducer
```
1 // key is a triangle (a, b, c)
2 // where a < b < c
3 // values = list of nulls
4 reduce(key, values) {
5 emit(key, null)
6 }
```

## Spark Solution

### Step 3: Create a Spark context object

In [3]:
from pyspark import SparkContext
sc = SparkContext() 
sc

### Step 4: Read graph via HDFS input

sample_graph.txt
```
1 2
2 3
2 4
2 5
3 4
4 5
```

In [4]:
lines = sc.textFile('sample_graph.txt', 1);

In [6]:
lines.collect()

[u'1 2', u'2 3', u'2 4', u'2 5', u'3 4', u'4 5']

### Step 5: Create all graph edges

(nodeA, nodeB) 의 출력을

- (nodeA, nodeB)
- (nodeB, nodeA)

In [7]:
def all_edges(s) :
    nodes = s.split(" ");
    start = long(nodes[0]);
    end = long(nodes[1]);
    list = []
    list.append( (start, end) )
    list.append( (end,   start) )
    return list

In [8]:
edges = lines.flatMap( all_edges )

In [9]:
edges.collect()

[(1L, 2L),
 (2L, 1L),
 (2L, 3L),
 (3L, 2L),
 (2L, 4L),
 (4L, 2L),
 (2L, 5L),
 (5L, 2L),
 (3L, 4L),
 (4L, 3L),
 (4L, 5L),
 (5L, 4L)]

### Step 6: Create RDD to generate triads

In [14]:
triads = edges.groupByKey();

In [27]:
debug1 = triads.collect();
for t1 in debug1 :
    print "debug1 t1[0] ", t1[0], ", t1[1] ", " ".join([str(x) for x in t1[1]] )

debug1 t1[0]  1 , t1[1]  2
debug1 t1[0]  2 , t1[1]  1 3 4 5
debug1 t1[0]  3 , t1[1]  2 4
debug1 t1[0]  4 , t1[1]  2 3 5
debug1 t1[0]  5 , t1[1]  2 4


### Step 7: Create all possible triads

In [32]:
def all_triads( s ) :
    # s[0] = Long (as a key)
    # s[1] = Iterable<Long> (as values)
    values = s[1]  
    result = [] 
    # Generate possible triads.
    for value in values :
        k2 = ( s[0], value )
        k2v2 = (k2, 0l);  # 직접 연결됨을 표시함.
        result.append( k2v2 )
    
    valuesCopy = []
    for item in values :
        valuesCopy.append( item )
        
    valuesCopy.sort()
    
    ## Generate possible triads.
    for i in range( len(valuesCopy)-1 ) :
        for j in range( i+1, len(valuesCopy) ) :
            k2 = (valuesCopy[i], valuesCopy[j] );
            k2v2 = ( k2, s[0] )
            result.append( k2v2 )
            
    return result

In [33]:
possibleTriads = triads.flatMap( all_triads  ) 

In [35]:
debug2 = possibleTriads.collect();
for t2 in debug2 :
    print "debug2 t2[0] =", t2[0], "t2[1]=", t2[1]


 debug2 t2[0] = (1L, 2L) t2[1]= 0
debug2 t2[0] = (2L, 1L) t2[1]= 0
debug2 t2[0] = (2L, 3L) t2[1]= 0
debug2 t2[0] = (2L, 4L) t2[1]= 0
debug2 t2[0] = (2L, 5L) t2[1]= 0
debug2 t2[0] = (1L, 3L) t2[1]= 2
debug2 t2[0] = (1L, 4L) t2[1]= 2
debug2 t2[0] = (1L, 5L) t2[1]= 2
debug2 t2[0] = (3L, 4L) t2[1]= 2
debug2 t2[0] = (3L, 5L) t2[1]= 2
debug2 t2[0] = (4L, 5L) t2[1]= 2
debug2 t2[0] = (3L, 2L) t2[1]= 0
debug2 t2[0] = (3L, 4L) t2[1]= 0
debug2 t2[0] = (2L, 4L) t2[1]= 3
debug2 t2[0] = (4L, 2L) t2[1]= 0
debug2 t2[0] = (4L, 3L) t2[1]= 0
debug2 t2[0] = (4L, 5L) t2[1]= 0
debug2 t2[0] = (2L, 3L) t2[1]= 4
debug2 t2[0] = (2L, 5L) t2[1]= 4
debug2 t2[0] = (3L, 5L) t2[1]= 4
debug2 t2[0] = (5L, 2L) t2[1]= 0
debug2 t2[0] = (5L, 4L) t2[1]= 0
debug2 t2[0] = (2L, 4L) t2[1]= 5


### Step 8: Create RDD to generate triangles

In [36]:
triadsGrouped = possibleTriads.groupByKey();

In [38]:
debug3 = triadsGrouped.collect();
for t3 in debug3 :
    print "debug3 t2[0] =", t3[0], "t3[1]=", " ".join([str(x) for x in t3[1]] )

debug3 t2[0] = (1L, 2L) t3[1]= 0
debug3 t2[0] = (3L, 2L) t3[1]= 0
debug3 t2[0] = (1L, 3L) t3[1]= 2
debug3 t2[0] = (4L, 5L) t3[1]= 2 0
debug3 t2[0] = (3L, 4L) t3[1]= 2 0
debug3 t2[0] = (5L, 4L) t3[1]= 0
debug3 t2[0] = (2L, 1L) t3[1]= 0
debug3 t2[0] = (1L, 5L) t3[1]= 2
debug3 t2[0] = (2L, 3L) t3[1]= 0 4
debug3 t2[0] = (1L, 4L) t3[1]= 2
debug3 t2[0] = (4L, 3L) t3[1]= 0
debug3 t2[0] = (4L, 2L) t3[1]= 0
debug3 t2[0] = (2L, 5L) t3[1]= 0 4
debug3 t2[0] = (5L, 2L) t3[1]= 0
debug3 t2[0] = (2L, 4L) t3[1]= 0 3 5
debug3 t2[0] = (3L, 5L) t3[1]= 2 4


### Step 9: Create all triangles

In [50]:
def all_triangles(s) :
    # s[0] = Tuple2<Long,Long> (as a key) = "<nodeA><,><nodeB>"
    # s[1] = Iterable<Long> (as values) = {0, n1, n2, n3, ...} or
    #                                     {n1, n2, n3, ...}
    key = s[0];
    values = s[1];
    
    mylist = []
    haveSeenSpecialNodeZero = False;
    for node in values :
        if 0L == node :
            haveSeenSpecialNodeZero = True
        else :
            mylist.append(node);
            
    result = []
    if haveSeenSpecialNodeZero :
        if len(mylist) == 0 :
            return result
        
        for node in mylist :
            aTriangle = [ key[0], key[1], node ]
            aTriangle.sort()
            t3 = ( aTriangle[0], aTriangle[1], aTriangle[2]  )
            result.append( t3 )
    
    return result

In [51]:
trianglesWithDuplicates = triadsGrouped.flatMap( all_triangles )

In [52]:
print "=== Triangles with Duplicates ==="
debug4 = trianglesWithDuplicates.collect();
for t4 in  debug4 :
    print "t4", t4

=== Triangles with Duplicates ===
t4 (2L, 4L, 5L)
t4 (2L, 3L, 4L)
t4 (2L, 3L, 4L)
t4 (2L, 4L, 5L)
t4 (2L, 3L, 4L)
t4 (2L, 4L, 5L)


### Step 10: Create unique triangles

In [53]:
uniqueTriangles = trianglesWithDuplicates.distinct();

In [54]:
uniqueTriangles.collect()

[(2L, 3L, 4L), (2L, 4L, 5L)]